In [21]:
import numpy as np
import open3d
from visual_utils import open3d_vis_utils as V

In [32]:
points = np.fromfile('/sda/dxg/OpenPCDet/data/kitti/testing/velodyne/000008.bin', dtype=np.float32, count=-1).reshape([-1,4])

In [14]:
V.draw_scenes(points=points)

In [24]:
from pcdet.utils.LiDAR_corruptions import rain_sim, snow_sim, fog_sim, scene_glare_noise

In [ ]:
points_rain = rain_sim(pointcloud=points, severity=5)

In [ ]:
points_snow = snow_sim(pointcloud=points, severity=5)

In [34]:
points_fog = fog_sim(pointcloud=points, severity=5)

In [36]:
points_glare = scene_glare_noise(pointcloud=points, severity=5)

In [20]:
%timeit points2 = snow_sim(pointcloud=points, severity=5)

8.69 s ± 74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from pcdet.utils.LiDAR_corruptions import moving_noise_bbox

In [43]:
import pickle
with open ('../data/kitti/kitti_infos_train.pkl', 'rb') as f:
    info = pickle.load(f)

In [45]:
info[0].keys()

dict_keys(['point_cloud', 'image', 'calib', 'annos'])

In [46]:
info[0]['point_cloud']

{'num_features': 4, 'lidar_idx': '000000'}

In [47]:
info[0]['annos']

{'name': array(['Pedestrian'], dtype='<U10'),
 'truncated': array([0.]),
 'occluded': array([0.]),
 'alpha': array([-0.2]),
 'bbox': array([[712.4 , 143.  , 810.73, 307.92]], dtype=float32),
 'dimensions': array([[1.2 , 1.89, 0.48]]),
 'location': array([[1.84, 1.47, 8.41]], dtype=float32),
 'rotation_y': array([0.01]),
 'score': array([-1.]),
 'difficulty': array([0], dtype=int32),
 'index': array([0], dtype=int32),
 'gt_boxes_lidar': array([[ 8.73138142, -1.85591745, -0.65469939,  1.2       ,  0.48      ,
          1.89      , -1.58079633]]),
 'num_points_in_gt': array([377], dtype=int32)}

In [33]:
V.draw_scenes(points=points)

In [37]:
V.draw_scenes(points=points_glare)

In [ ]:
from pcdet.utils.Camera_corruptions import ImageBBoxOperation
import torch
from skimage import io


img = io.imread('/sda/dxg/OpenPCDet/data/kitti/training/image_2/000008.png')
severity = 5
bbox_shear = ImageBBoxOperation(severity)
img_bgr_255_np_uint8 = img
bboxes_corners = results['gt_bboxes_3d'].corners
bboxes_centers = results['gt_bboxes_3d'].center
lidar2img = results['lidar2img'] 
img_rgb_255_np_uint8 = img_bgr_255_np_uint8[:, :, [2, 1, 0]]
c = [0.05, 0.1, 0.15, 0.2, 0.25][bbox_shear.severity - 1]
b = np.random.uniform(c - 0.05, c + 0.05) * np.random.choice([-1, 1])
d = np.random.uniform(c - 0.05, c + 0.05) * np.random.choice([-1, 1])
e = np.random.uniform(c - 0.05, c + 0.05) * np.random.choice([-1, 1])
f = np.random.uniform(c - 0.05, c + 0.05) * np.random.choice([-1, 1])
transform_matrix = torch.tensor([
    [1, 0, b],
    [d, 1, e],
    [f, 0, 1]
]).float()

image_aug_rgb = bbox_shear(
    image=img_rgb_255_np_uint8,
    bboxes_centers=bboxes_centers,
    bboxes_corners=bboxes_corners,
    transform_matrix=transform_matrix,
    lidar2img=lidar2img,
)
image_aug_bgr = image_aug_rgb[:, :, [2, 1, 0]]
results['img'] = image_aug_bgr

In [6]:
import os
# import pathlib2
path='../output'
dataset = 'kitti'
if dataset == 'kitti':
    dataset = 'kitti_models'
model = 'pointpillar'

dir = os.path.join(path, dataset, model)
for corruption in os.listdir(dir):
    dir2 = os.path.join(dir, corruption, 'eval')
    for epoch in os.listdir(dir2):
        dir3 = os.path.join(dir2, epoch, 'val', 'default')
        for log in os.listdir(dir3):
            if 'pkl' in log:
                continue
            print(log)

log_eval_[None, None]_0_20230609-102337.txt
log_eval_[None, None]_0_20230609-103018.txt


In [1]:
path / dataset / model

NameError: name 'path' is not defined